In [1]:
import os
from dotenv import load_dotenv
from typing import Any

from azure.identity import AzureCliCredential
from agent_framework.azure import AzureOpenAIResponsesClient

from agent_framework import Message, WorkflowEvent
from agent_framework.orchestrations import ConcurrentBuilder

In [2]:
load_dotenv(override=True)

project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model = os.getenv("AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME")

print("Project Endpoint: ", project_endpoint)
print("Model: ", model)

Project Endpoint:  https://jdramkumar32-1674-resource.services.ai.azure.com/api/projects/jdramkumar32-1674
Model:  gpt-4o


In [3]:
credential = AzureCliCredential()
chat_client = AzureOpenAIResponsesClient(
    project_endpoint=project_endpoint,
    deployment_name=model,
    credential=credential,
)

In [4]:
researcher = chat_client.as_agent(
    instructions=(
        "You're an expert market and product researcher. Given a prompt, provide concise, factual insights,"
        " opportunities, and risks."
    ),
    name="researcher",
)

marketer = chat_client.as_agent(
    instructions=(
        "You're a creative marketing strategist. Craft compelling value propositions and target messaging"
        " aligned to the prompt."
    ),
    name="marketer",
)

legal = chat_client.as_agent(
    instructions=(
        "You're a cautious legal/compliance reviewer. Highlight constraints, disclaimers, and policy concerns"
        " based on the prompt."
    ),
    name="legal",
)

In [5]:
# Define a custom aggregator callback that uses the chat client to summarize
async def summarize_results(results: list[Any]) -> str:
    # Extract one final assistant message per agent
    expert_sections: list[str] = []
    for r in results:
        try:
            # r is an AgentExecutorResponse; use full_conversation to get messages
            conversation = getattr(r, "full_conversation", [])
            assistant_msgs = [m for m in conversation if m.role == "assistant"]
            final_text = assistant_msgs[-1].text if assistant_msgs else "(no content)"
            expert_sections.append(f"{getattr(r, 'executor_id', 'expert')}:\n{final_text}")
        except Exception as e:
            expert_sections.append(f"{getattr(r, 'executor_id', 'expert')}: (error: {type(e).__name__}: {e})")

    # Ask the model to synthesize a concise summary of the experts' outputs
    system_msg = Message(
        role="system",
        contents=[
            """
            You are a helpful assistant that consolidates multiple domain expert outputs into one cohesive, 
            concise summary with clear takeaways. Keep it under 200 words.
            """],
    )
    user_msg = Message(role="user", contents=["\n\n".join(expert_sections)])

    response = await chat_client.get_response([system_msg, user_msg])
    # Return the model's final assistant text as the completion result
    return response.messages[-1].text if response.messages else ""


In [6]:
workflow = (
    ConcurrentBuilder(participants=[researcher, marketer, legal])
    .with_aggregator(summarize_results)
    .build()
)

In [7]:
output_evt: WorkflowEvent | None = None
query = "We are launching a new budget-friendly electric bike for urban and rural commuters."

async for event in workflow.run(query, stream=True):
    if event.type == "output":
        output_evt = event

In [8]:
if output_evt:
    print("===== Final Consolidated Output =====")
    print(output_evt.data)

===== Final Consolidated Output =====
The electric bike market is seeing significant growth driven by heightened environmental awareness and urbanization. Key demographics include young professionals, eco-conscious consumers, and rural commuters where public transport is limited. Essential features for a successful product include long battery life, affordability, lightweight design, and low maintenance. Opportunities lie in leveraging government subsidies, forming partnerships to promote eco-friendly transportation, and engaging communities through bike-sharing programs. However, challenges such as market saturation, inadequate infrastructure, and regulatory changes pose risks.

The proposed value proposition emphasizes affordability and efficiency, catering to urban commuters with features like a compact design and quick charging, and to rural commuters with a durable build and long battery life. Target messaging highlights the environmental and cost benefits.

From a legal perspecti